In [ ]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv("Data/train.csv")
test = pd.read_csv("Data/test.csv")

I am using excel for other insightful analysis

### Treatment

Treating Initial Data : 

In [ ]:
train.isnull().sum()

Changing Label of the Target Column

In [ ]:
train['Loan_Status']=train['Loan_Status'].apply( lambda x : 1 if x=='Y' else 0)

Treating Missing Data 

In [ ]:
train['Dependents'].fillna('0',inplace=True)

In [ ]:
train['Gender'].fillna('Unknown',inplace=True)
train['Married'].fillna('Yes',inplace=True)
train['Self_Employed'].fillna('Unknown',inplace=True)
train['Credit_History'].fillna(-1,inplace=True)

In [ ]:
train['Loan_Amount_Term'].fillna(train['Loan_Amount_Term'].median(skipna=True),inplace=True)
train['LoanAmount'].fillna(train['LoanAmount'].median(skipna=True),inplace=True)

### Feature Engineering 

If Co-Applicant Earns. Both Members Earn

In [ ]:
train['BothEarn'] = (train.CoapplicantIncome!=0).astype('int') 
train['BothEarn'] = train['BothEarn'].map({1:'Yes',0:'No'})

Total Family Income

In [ ]:
train['TotalIncome'] = train['ApplicantIncome']+train['CoapplicantIncome']

Family Size

In [ ]:
c=0 #counter
fam = train['Dependents'].copy()

for i in train['BothEarn']:
    if train.loc[c,'Married']=='Yes':
        fam.loc[c] = '2+'+fam.loc[c]
    else :
        fam.loc[c] = '1+'+fam.loc[c]
    c+=1

# fam = "2+"+train['Dependents']
train['FamilySize'] = fam.apply(lambda x: eval(x[:-1]) if x[-1]=='+' else eval(x))

Income per person in family

In [ ]:
train['IncomePerMember'] = train['TotalIncome']/train['FamilySize']

Income Strength Binning

In [ ]:
bins = np.linspace(train['IncomePerMember'].min(),train['IncomePerMember'].max(),25,True)
bins = np.append(bins[0:4],train['IncomePerMember'].max())

train['IncomeStrength'] = pd.cut(train['IncomePerMember'],bins=bins,labels=['low','Med','High','V. High'],include_lowest=True)

In [ ]:
plt.figure(figsize=(7,5))
plot_data = pd.pivot_table(data=train,index='IncomeStrength',values='Loan_Status')
sns.barplot(plot_data.index,'Loan_Status',data=plot_data)
plt.show()

Monthly Installment (without interest - Just to estimate)

In [ ]:
train['MonthlyInstal'] = train['LoanAmount']/train['Loan_Amount_Term']

## EDA

### Study

In [ ]:
# a = pd.pivot_table(train,train.columns,'Loan_Status')
# a

In [ ]:
# b = pd.pivot_table(train,train.columns,'Loan_Status',aggfunc=['median'])
# b

In [ ]:
# b = pd.pivot_table(train,train.columns,'Loan_Status',aggfunc=['min'])
# b

In [ ]:
# b = pd.pivot_table(train,train.columns,'Loan_Status',aggfunc=['max'])
# b

In [ ]:
# fig, axs = plt.subplots(8,2,figsize=(30,100))
# fig.subplots_adjust(hspace = .5, wspace=.5)
# axs = axs.ravel()
# counter=0

# for i in ['ApplicantIncome','CoapplicantIncome', 'FamilySize',
# 'IncomePerMember', 'LoanAmount', 'Loan_Amount_Term', 'MonthlyInstal','TotalIncome']:

#     bins = ([train[i].min(),train[i].quantile(.10),train[i].quantile(.3),train[i].quantile(.6),train[i].quantile(.9),train[i].max()])
#     bins = [round(i) for i in bins]
#     if bins[0]==0 : bins.insert(1,1)  #no matter the distribution, if it begins with 0, make a 1
#     bins = list(set(bins))
#     bins.sort()
    
#     data_pivot = train[['Loan_Status']]
#     data_pivot[i] = pd.cut(train[i],bins=bins,include_lowest=True)
#     data_plot = pd.pivot_table(data_pivot,'Loan_Status',i)

#     # print(data_pivot[i].value_counts()) #debugging
#     # print(data_plot) #debugging

#     axs[counter].set_title(i)
#     sns.barplot(data_plot.index,'Loan_Status',data=data_plot,ax=axs[counter])
#     axs[counter].set_ylabel("Percentage of Approval")
#     axs[counter].set_xticklabels(data_plot.index,rotation=90)
#     counter+=1   


#     # print(bins) #debugging

#     axs[counter].set_title(i)
#     # axs[counter].hist(train[i],bins=bins)
#     sns.distplot(train[i],bins=bins,ax=axs[counter],kde=False)
#     axs[counter].set_ylabel("Distribution at Each Label")
#     # axs[counter].set_xlabel(bins,rotation=90)
#     counter+=1

#### Insights 

1. __Applicant Income__ : slight
1. __Co-applicant Income__ : slight
1. __Family Size__ : Not Significant
1. __Income Per Member__ : slight
1. __Loan Amount__ : slight
1. __Loan Amount Term__ : slight
1. __Monthly Installment__ : Not Significant
1. __Total Income__ :  slight


### Checking Significant Variables

In [ ]:
# i='TotalIncome'
# data_pivot = train[['Loan_Status']]
# data_pivot[i] = pd.qcut(train[i],q=20)
# data_plot = pd.pivot_table(data_pivot,'Loan_Status',i,aggfunc=['mean','count'])
# print(data_plot)

In [ ]:
# i='CoapplicantIncome'
# data_pivot = train[['Loan_Status']]
# data_pivot[i] = pd.qcut(train[i],q=20,duplicates='drop')
# data_plot = pd.pivot_table(data_pivot,'Loan_Status',i,aggfunc=['mean','count'])
# print(data_plot)

In [ ]:
# i='IncomePerMember'
# data_pivot = train[['Loan_Status']]
# data_pivot[i] = pd.qcut(train[i],q=20,duplicates='drop')
# data_plot = pd.pivot_table(data_pivot,'Loan_Status',i,aggfunc=['mean','count'])
# print(data_plot)

In [ ]:
# i='LoanAmount'
# data_pivot = train[['Loan_Status']]
# data_pivot[i] = pd.qcut(train[i],q=20,duplicates='drop')
# data_plot = pd.pivot_table(data_pivot,'Loan_Status',i,aggfunc=['mean','count'])
# print(data_plot)

In [ ]:
# i='Loan_Amount_Term'
# data_pivot = train[['Loan_Status']]
# data_pivot[i] = pd.qcut(train[i],q=20,duplicates='drop')
# data_plot = pd.pivot_table(data_pivot,'Loan_Status',i,aggfunc=['mean','count'])
# print(data_plot)

In [ ]:
# i='TotalIncome'
# data_pivot = train[['Loan_Status']]
# data_pivot[i] = pd.qcut(train[i],q=20,duplicates='drop')
# data_plot = pd.pivot_table(data_pivot,'Loan_Status',i,aggfunc=['mean','count'])
# print(data_plot)

### Categorical

In [ ]:
# cat_var = ['Gender', 'Married', 'Dependents', 'Education',
#        'Self_Employed', 'Property_Area', 'BothEarn', 'IncomeStrength','Credit_History']

In [ ]:
# fig, axs = plt.subplots(9,2,figsize=(30,100))
# fig.subplots_adjust(hspace = .5, wspace=.5)
# axs = axs.ravel()
# counter=0

# for i in cat_var:
    
#     data_pivot = train[[i,'Loan_Status']]
#     data_plot = pd.pivot_table(data_pivot,'Loan_Status',i)
# #     print(data_plot)
    
#     axs[counter].set_title(i)
#     sns.barplot(data_plot.index,'Loan_Status',data=data_plot,ax=axs[counter])
#     axs[counter].set_ylabel("Percentage of Approval")
#     axs[counter].set_xticklabels(data_plot.index,rotation=90)
#     counter+=1   


#     # print(bins) #debugging

#     axs[counter].set_title(i)
#     axs[counter].hist(train[i])
#     # sns.distplot(train[i],ax=axs[counter],kde=False)
#     axs[counter].set_ylabel("Distribution at Each Label")
#     # axs[counter].set_xlabel(bins,rotation=90)
#     counter+=1

## END EDA

### Data Selection

In [ ]:
# #binning is done as values observed from Graph
# temp = pd.DataFrame()

# temp['Loan_Amount_Term']=pd.cut(train['Loan_Amount_Term'],bins=[0,360,500])

# temp['ApplicantIncome'] = pd.cut(train['ApplicantIncome'],bins=[0,210,63337,81000])

# temp['CoapplicantIncome'] = pd.cut(train['CoapplicantIncome'],bins=[0,20000,42000])

# temp['IncomePerMember'] = pd.cut(train['IncomePerMember'],bins=[0,950,10000,36000])

# temp['LoanAmount'] = pd.cut(train['LoanAmount'], bins=[0,57,87,188,700])

In [ ]:
#binning is done as values observed from Graph
temp = pd.DataFrame()

temp['Loan_Amount_Term']=(train['Loan_Amount_Term'])
temp['ApplicantIncome'] =(train['ApplicantIncome'])
temp['CoapplicantIncome'] =(train['CoapplicantIncome'])
temp['IncomePerMember'] =(train['IncomePerMember'])
temp['LoanAmount'] =(train['LoanAmount'])

In [ ]:
cat_cols =['Credit_History','Married','Education','Property_Area']

In [ ]:
# cont_cols =['TotalIncome','Loan_Amount_Term','ApplicantIncome','CoapplicantIncome','IncomePerMember','LoanAmount']

In [ ]:
temp = pd.concat([temp,train[cat_cols]],axis=1)

In [ ]:
# cat_cols = temp.columns[(temp.dtypes == 'object') | (temp.dtypes == 'category')]
# cont_cols = temp.columns[(temp.dtypes != 'object') & (temp.dtypes != 'category')]

## Test Train Split

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

temp = pd.get_dummies(temp)
X = pd.DataFrame(sc.fit_transform(temp),columns=temp.columns)
# X = pd.get_dummies(temp)
y = train.Loan_Status


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=100)

### SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
oversample = SMOTE()
X_train,y_train = oversample.fit_resample(X_train,y_train)

## Model Development

In [ ]:
from sklearn import metrics

#### RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train,y_train)

y_hat = rf.predict(X_test)

rf.score(X_test,y_test)

#### LGBM

In [ ]:
# from lightgbm import LGBMClassifier

# LGBM = LGBMClassifier()
# LGBM.fit(X_train, y_train)

# yhat = LGBM.predict(X_test)

# print(metrics.accuracy_score(y_test, yhat))

#### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(X_train,y_train)

yhat = model.predict(X_test)

print(metrics.accuracy_score(y_test, yhat))

In [ ]:
from sklearn.linear_model import LogisticRegression

lor = LogisticRegression()
lor.fit(X_train,y_train)

yhat = lor.predict(X_test)

print(metrics.accuracy_score(y_test, yhat))

#### Kmeans

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
Ks = 1 #Kstart
Ke = 15 #Kstop
mean_acc = np.zeros((Ke-Ks+1))

for n in range(Ks,Ke+1):
      
    KNN = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yhat=KNN.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)

mean_acc

In [ ]:
KNN = KNeighborsClassifier(n_neighbors = 4).fit(X_train,y_train)
yhat=KNN.predict(X_test)
metrics.accuracy_score(y_test, yhat)

# Prediction

### Test Data Treatment

In [ ]:
test.isnull().sum()

Treating Missing Data 

In [ ]:
test['Dependents'].fillna('0',inplace=True)

test['Gender'].fillna('Unknown',inplace=True)
test['Married'].fillna('Unknown',inplace=True)
test['Self_Employed'].fillna('Unknown',inplace=True)
test['Credit_History'].fillna(99,inplace=True)

test['Loan_Amount_Term'].fillna(test['Loan_Amount_Term'].median(skipna=True),inplace=True)
test['LoanAmount'].fillna(test['LoanAmount'].median(skipna=True),inplace=True)

### Feature Engineering 

In [ ]:
#If Co-Applicant Earns. Both Members Earn

test['BothEarn'] = (test.CoapplicantIncome!=0).astype('int') 
test['BothEarn'] = test['BothEarn'].map({1:'Yes',0:'No'})

#Total Family Income

test['TotalIncome'] = test['ApplicantIncome']+test['CoapplicantIncome']

#Family Size

c=0 #counter
fam = test['Dependents'].copy()

for i in test['BothEarn']:
    if test.loc[c,'Married']=='Yes':
        fam.loc[c] = '2+'+fam.loc[c]
    else :
        fam.loc[c] = '1+'+fam.loc[c]
    c+=1

# fam = "2+"+test['Dependents']
test['FamilySize'] = fam.apply(lambda x: eval(x[:-1]) if x[-1]=='+' else eval(x))

#Income per person in family

test['IncomePerMember'] = test['TotalIncome']/test['FamilySize']

#Income Strength Binning

bins = np.linspace(test['IncomePerMember'].min(),test['IncomePerMember'].max(),25,True)
bins = np.append(bins[0:4],test['IncomePerMember'].max())

test['IncomeStrength'] = pd.cut(test['IncomePerMember'],bins=bins,labels=['low','Med','High','V. High'],include_lowest=True)


#Monthly Installment (without interest - Just to estimate)

test['MonthlyInstal'] = test['LoanAmount']/test['Loan_Amount_Term']

In [ ]:
#binning is done as values observed from Graph
temp_test = pd.DataFrame()

temp_test['Loan_Amount_Term']=pd.cut(test['Loan_Amount_Term'],bins=[0,360,500])

temp_test['ApplicantIncome'] = pd.cut(test['ApplicantIncome'],bins=[0,210,63337,81000])

temp_test['CoapplicantIncome'] = pd.cut(test['CoapplicantIncome'],bins=[0,20000,42000])

temp_test['IncomePerMember'] = pd.cut(test['IncomePerMember'],bins=[0,950,10000,36000])

temp_test['LoanAmount'] = pd.cut(test['LoanAmount'], bins=[0,57,87,188,700])

In [ ]:
cat_cols =['Credit_History','Married','Education','Property_Area']

In [ ]:
temp_test = pd.concat([temp_test,test[cat_cols]],axis=1)

In [ ]:
temp_test = pd.get_dummies(temp_test)
temp_test = pd.DataFrame(sc.fit_transform(temp_test),columns=temp_test.columns)

In [ ]:
y_hat = KNN.predict(temp_test)

In [ ]:
sol=pd.DataFrame()
sol['Loan_Status']=y_hat
sol['Loan_ID']=test['Loan_ID']
sol.set_index('Loan_ID',inplace=True)
sol['Loan_Status'] = sol['Loan_Status'].map({1:'Y',0:'N'})

In [ ]:
sol.to_csv('Submission_KNN.csv')